In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class EZReaderDataset(Dataset):
    """
    Loads the merged EZ-Reader + Provo data and returns one sentence at a time.
    Expects a CSV with columns:
      sentence_id, word_in_sentence,
      freq_z, len_z, cloze_z,
      SFD_z, FFD_z, GD_z, TT_z, GP_z,
      PrF, Pr1, Pr2, PrS
    """
    def __init__(self, path_csv, device=device):
        df = pd.read_csv(path_csv)
       
        self.sent_ids = sorted(df['sentence_id'].unique())
        self.groups = {
            sid: df[df['sentence_id']==sid]
                     .sort_values('word_in_sentence')
                     for sid in self.sent_ids
        }
        self.device = device

    def __len__(self):
        return len(self.sent_ids)

    def __getitem__(self, idx):
        sid = self.sent_ids[idx]
        g = self.groups[sid]
        # inputs: freq_z, len_z, cloze_z
        X = torch.tensor(
            g[['freq_z','len_z','cloze_z']].values,
            dtype=torch.float32,
            device=self.device
        )            
        # continuous outputs: SFD_z, FFD_z, GD_z, TT_z, GP_z
        y_cont = torch.tensor(
            g[['SFD_z','FFD_z','GD_z','TT_z','GP_z']].values,
            dtype=torch.float32,
            device=self.device
        )            
        # probabilistic outputs: PrF, Pr1, Pr2, PrS
        y_prob = torch.tensor(
            g[['PrF','Pr1','Pr2','PrS']].values,
            dtype=torch.float32,
            device=self.device
        )            
        return X, y_cont, y_prob

def collate_sentences(batch):
    """
    Pads a batch of (X, y_cont, y_prob) by sequence length with zeros,
    and returns lengths on the same device as X.
    """

    Xs, Yc, Yp = zip(*batch)
   
    device = Xs[0].device

    lengths = torch.tensor([x.size(0) for x in Xs],
                           dtype=torch.long,
                           device=device)

    T_max = lengths.max().item()

    def pad(seqs, dim):
        padded = torch.zeros(len(seqs), T_max, dim, device=device)
        for i, seq in enumerate(seqs):
            padded[i, : seq.size(0), :] = seq
        return padded

    X_pad  = pad(Xs, 3)
    yc_pad = pad(Yc, 5)
    yp_pad = pad(Yp, 4)
    return X_pad, yc_pad, yp_pad, lengths


class EZReaderRNN(nn.Module):
    """
    RNN that mimics EZ-Reader’s serial processing:
     - uni-directional LSTM over word-by-word inputs
     - two heads: regression for durations, sigmoid‐prob for fixation‐probs
    """
    def __init__(self,
                 input_size=3,
                 hidden_size=256,
                 num_layers=1,
                 cont_out=5,
                 prob_out=4,
                 dropout=0.0):
        super().__init__()
        self.lstm = nn.LSTM(input_size,
                            hidden_size,
                            num_layers,
                            batch_first=True,
                            dropout=dropout)
        
        self.reg_head = nn.Linear(hidden_size, cont_out)
        
        self.prob_head = nn.Sequential(
            nn.Linear(hidden_size, prob_out),
            nn.Sigmoid()
        )

    def forward(self, x, lengths):

        packed = nn.utils.rnn.pack_padded_sequence(
            x, lengths.cpu(), batch_first=True, enforce_sorted=False
        )
        packed_out, _ = self.lstm(packed)
        out, _ = nn.utils.rnn.pad_packed_sequence(
            packed_out, batch_first=True
        ) 

        
        y_cont = self.reg_head(out)     
        y_prob = self.prob_head(out)   
        return y_cont, y_prob


if __name__ == "__main__":
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    ds = EZReaderDataset('ezreader_training_data_with_ids.csv', device=device)
    dl = DataLoader(ds,
                    batch_size=16,
                    shuffle=True,
                    collate_fn=collate_sentences)

    model = EZReaderRNN().to(device)
    # define losses & optimizer
    mse_loss   = nn.MSELoss()
    bce_loss   = nn.BCELoss()
    optimizer  = torch.optim.Adam(model.parameters(), lr=1e-3)

  
    for Xb, Yc_b, Yp_b, Lb in dl:
        optimizer.zero_grad()
        pred_cont, pred_prob = model(Xb, Lb)


        device = pred_cont.device
        seq_len = pred_cont.size(1)
        mask = (torch.arange(seq_len, device=device)[None, :]
                < Lb[:, None]).float().unsqueeze(-1)

        loss_cont = mse_loss(pred_cont * mask, Yc_b * mask)
        loss_prob = bce_loss(pred_prob * mask, Yp_b * mask)
        (loss_cont + loss_prob).backward()
        optimizer.step()
        print(f"batch loss: {loss_cont.item() + loss_prob.item():.4f}")
        break


In [ ]:
from torch.utils.data import random_split

full_ds = EZReaderDataset('ezreader_training_data_with_ids.csv', device=device)
n = len(full_ds)
n_train = int(0.8 * n)
n_val   = int(0.1 * n)
n_test  = n - n_train - n_val

train_ds, val_ds, test_ds = random_split(full_ds, [n_train, n_val, n_test])

train_loader = DataLoader(train_ds, batch_size=16, shuffle=True,  collate_fn=collate_sentences)
val_loader   = DataLoader(val_ds,   batch_size=16, shuffle=False, collate_fn=collate_sentences)
test_loader  = DataLoader(test_ds,  batch_size=16, shuffle=False, collate_fn=collate_sentences)


In [ ]:
import torch
import numpy as np
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr


def evaluate(model, loader):
    model.eval()
    all_true_cont, all_pred_cont = [], []
    all_true_prob, all_pred_prob = [], []
    with torch.no_grad():
        for Xb, Yc_b, Yp_b, Lb in loader:
            pred_c, pred_p = model(Xb, Lb)
            device = pred_c.device

            # build mask
            seq_len = pred_c.size(1)
            mask = (torch.arange(seq_len, device=device)[None, :]
                    < Lb[:,None]).unsqueeze(-1)

            # extract only valid timesteps
            true_c = (Yc_b * mask).reshape(-1, pred_c.size(2))[mask.reshape(-1)]
            pred_c = (pred_c * mask).reshape(-1, pred_c.size(2))[mask.reshape(-1)]
            true_p = (Yp_b * mask).reshape(-1, pred_p.size(2))[mask.reshape(-1)]
            pred_p = (pred_p * mask).reshape(-1, pred_p.size(2))[mask.reshape(-1)]

            all_true_cont.append(true_c.cpu().numpy())
            all_pred_cont.append(pred_c.cpu().numpy())
            all_true_prob.append(true_p.cpu().numpy())
            all_pred_prob.append(pred_p.cpu().numpy())

    true_cont = np.vstack(all_true_cont)
    pred_cont = np.vstack(all_pred_cont)
    true_prob = np.vstack(all_true_prob)
    pred_prob = np.vstack(all_pred_prob)

    # Continuous metrics
    rmse = np.sqrt(mean_squared_error(true_cont.flatten(), pred_cont.flatten()))
    r, _  = pearsonr(true_cont.flatten(), pred_cont.flatten())

    # Probabilities as regression
    mse_prob = mean_squared_error(true_prob.flatten(), pred_prob.flatten())

    return rmse, r, mse_prob


# 1. Initialize checkpoint variables
best_val_rmse = float('inf')
best_epoch   = -1

# 2. Training + validation loop
num_epochs = 50
for epoch in range(1, num_epochs+1):
    # --- training ---
    model.train()
    train_losses = []
    for Xb, Yc_b, Yp_b, Lb in train_loader:
        optimizer.zero_grad()
        pred_c, pred_p = model(Xb, Lb)
        device = pred_c.device
        seq_len = pred_c.size(1)
        mask = (torch.arange(seq_len, device=device)[None, :] < Lb[:,None]) \
                   .float().unsqueeze(-1)

        loss_c = mse_loss(pred_c * mask, Yc_b * mask)
        loss_p = bce_loss(pred_p * mask, Yp_b * mask)
        loss = loss_c + loss_p
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())

    avg_train_loss = np.mean(train_losses)

    # --- evaluation on validation set ---
    model.eval()
    all_true_cont, all_pred_cont = [], []
    all_true_prob, all_pred_prob = [], []
    with torch.no_grad():
        for Xb, Yc_b, Yp_b, Lb in val_loader:
            pred_c, pred_p = model(Xb, Lb)
            device = pred_c.device
            seq_len = pred_c.size(1)
            mask = (torch.arange(seq_len, device=device)[None, :] < Lb[:,None]) \
                       .unsqueeze(-1)

            # flatten valid timesteps
            true_c = (Yc_b * mask).reshape(-1, pred_c.size(2))[mask.reshape(-1)]
            pred_c = (pred_c * mask).reshape(-1, pred_c.size(2))[mask.reshape(-1)]
            true_p = (Yp_b * mask).reshape(-1, pred_p.size(2))[mask.reshape(-1)]
            pred_p = (pred_p * mask).reshape(-1, pred_p.size(2))[mask.reshape(-1)]

            all_true_cont.append(true_c.cpu().numpy())
            all_pred_cont.append(pred_c.cpu().numpy())
            all_true_prob.append(true_p.cpu().numpy())
            all_pred_prob.append(pred_p.cpu().numpy())

    true_cont = np.vstack(all_true_cont)
    pred_cont = np.vstack(all_pred_cont)
    true_prob = np.vstack(all_true_prob)
    pred_prob = np.vstack(all_pred_prob)

    # compute metrics
    val_rmse     = np.sqrt(mean_squared_error(true_cont.flatten(), pred_cont.flatten()))
    val_r, _     = pearsonr(true_cont.flatten(), pred_cont.flatten())
    val_mseprob  = mean_squared_error(true_prob.flatten(), pred_prob.flatten())

    print(f"Epoch {epoch:02d} | train loss {avg_train_loss:.4f}"
          f" | val RMSE {val_rmse:.4f}, r {val_r:.3f}, prob-MSE {val_mseprob:.4f}")

    # 3. Checkpoint 
    if val_rmse < best_val_rmse:
        best_val_rmse = val_rmse
        best_epoch    = epoch
        torch.save(model.state_dict(), 'best_model.pt')
        

# 4. After training
print(f"\nTraining complete. Best validation RMSE {best_val_rmse:.4f} at epoch {best_epoch}.")
print("Best model saved to best_model.pt")



In [ ]:
# Instantiate and load the trained model
model = EZReaderRNN().to(device)
model.load_state_dict(torch.load('best_model.pt', map_location=device))
model.eval()

# === Evaluation on Test Set ===

all_true_c, all_pred_c = [], []
all_true_p, all_pred_p = [], []

with torch.no_grad():
    for Xb, Yc_b, Yp_b, Lb in test_loader:
        pc, pp = model(Xb, Lb)
        device = pc.device
        seq_len = pc.size(1)
        mask = (torch.arange(seq_len, device=device)[None, :] < Lb[:,None]).unsqueeze(-1)

        true_c = (Yc_b * mask).reshape(-1, pc.size(2))[mask.reshape(-1)]
        pred_c = (pc   * mask).reshape(-1, pc.size(2))[mask.reshape(-1)]
        true_p = (Yp_b * mask).reshape(-1, pp.size(2))[mask.reshape(-1)]
        pred_p = (pp   * mask).reshape(-1, pp.size(2))[mask.reshape(-1)]

        all_true_c.append(true_c.cpu().numpy())
        all_pred_c.append(pred_c.cpu().numpy())
        all_true_p.append(true_p.cpu().numpy())
        all_pred_p.append(pred_p.cpu().numpy())

true_c = np.vstack(all_true_c)
pred_c = np.vstack(all_pred_c)
true_p = np.vstack(all_true_p)
pred_p = np.vstack(all_pred_p)

# Metrics
rmse = np.sqrt(mean_squared_error(true_c.flatten(), pred_c.flatten()))
r, _ = pearsonr(true_c.flatten(), pred_c.flatten())
prob_mse = mean_squared_error(true_p.flatten(), pred_p.flatten())

print(f"Test RMSE (durations, z‐scores): {rmse:.4f}")
print(f"Test Pearson's r (durations):    {r:.4f}")
print(f"Test MSE (probabilities):         {prob_mse:.4f}")

In [ ]:
import os
import zipfile
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler

# === Paths ===
PROVO_CSV    = 'Provo_Corpus-Eyetracking_Data.csv'
TRAIN_CSV    = 'ezreader_training_data_with_ids.csv'
BEST_MODEL   = 'best_model.pt'
OUT_CSV      = 'RNN_Word-Based_Means.csv'

# === 1. compute Provo aggregates
df_p = pd.read_csv(PROVO_CSV)
agg = (
    df_p.groupby(['Text_ID','Word_In_Sentence_Number'])
        .agg(
            SFD=('IA_DWELL_TIME',              lambda x: x[df_p.loc[x.index,'IA_FIXATION_COUNT']==1].mean()),
            FFD=('IA_FIRST_FIXATION_DURATION', 'mean'),
            GD=('IA_FIRST_RUN_DWELL_TIME',     'mean'),
            TT=('IA_DWELL_TIME',               'mean'),
            GP=('IA_RUN_COUNT',                lambda x: np.mean(x>0)),
        )
        .reset_index()
        .rename(columns={'Text_ID':'sentence_id',
                         'Word_In_Sentence_Number':'word_in_sentence'})
)
agg = agg.sort_values(['sentence_id','word_in_sentence']).reset_index(drop=True)

# Fit scaler on continuous ground truth
cont_cols = ['SFD','FFD','GD','TT','GP']
scaler_y_cont = StandardScaler().fit(agg[cont_cols].values)

# === 2. Load simulation map for words and positions ===

sim_file = 'simulation_corpus_final.txt'

# Load simulation corpus
sim_cols = ['frequency','length','cloze','word_raw']
df_sim = pd.read_csv(sim_file, delim_whitespace=True, names=sim_cols, engine='python')
df_sim['is_end'] = df_sim['word_raw'].str.endswith('@')
df_sim['word'] = df_sim['word_raw'].str.rstrip('@')
df_sim['sentence_id'] = df_sim['is_end'].cumsum().astype(int)
df_sim['word_in_sentence'] = df_sim.groupby('sentence_id').cumcount() + 1


df_map = df_sim[['sentence_id','word_in_sentence','word']]

# === 3. Load training CSV for inference
df_train = pd.read_csv(TRAIN_CSV)

groups = {
    sid: grp.sort_values('word_in_sentence')
    for sid, grp in df_train.groupby('sentence_id')
}

# === 4. Load model ===
class EZReaderRNN(nn.Module):
    def __init__(self, input_size=3, hidden_size=256, num_layers=1,
                 cont_out=5, prob_out=4, dropout=0.0):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers,
                            batch_first=True, dropout=dropout)
        self.reg_head = nn.Linear(hidden_size, cont_out)
        self.prob_head = nn.Sequential(nn.Linear(hidden_size, prob_out), 
                                       nn.Sigmoid())

    def forward(self, x, lengths):
        packed = nn.utils.rnn.pack_padded_sequence(
            x, lengths.cpu(), batch_first=True, enforce_sorted=False
        )
        out_pack, _ = self.lstm(packed)
        out, _ = nn.utils.rnn.pad_packed_sequence(out_pack, batch_first=True)
        return self.reg_head(out), self.prob_head(out)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = EZReaderRNN().to(device)
model.load_state_dict(torch.load(BEST_MODEL, map_location=device))
model.eval()

# === 5. Inference ===
pred_cont_z = []
pred_prob = []
sid_list = []
wordpos_list = []

with torch.no_grad():
    for sid, grp in sorted(groups.items()):
       
        X = torch.tensor(grp[['freq_z','len_z','cloze_z']].values,
                         dtype=torch.float32, device=device)
        lengths = torch.tensor([X.size(0)], dtype=torch.long, device=device)
        X = X.unsqueeze(0) 
        # Forward pass
        pc_z, pp = model(X, lengths)
        pc_z = pc_z.squeeze(0).cpu().numpy()  
        pp = pp.squeeze(0).cpu().numpy()      
        # Record predictions
        for i in range(pc_z.shape[0]):
            sid_list.append(sid)
            wordpos_list.append(int(grp.iloc[i]['word_in_sentence']))
            pred_cont_z.append(pc_z[i])
            pred_prob.append(pp[i])

# Convert lists to arrays
pred_cont_z = np.vstack(pred_cont_z)  
pred_prob = np.vstack(pred_prob)     

# 6. Inverse transform continuous predictions to ms
pred_cont = scaler_y_cont.inverse_transform(pred_cont_z)

# 7. Assemble output DataFrame
out_df = pd.DataFrame(pred_cont, columns=cont_cols)
prob_cols = ['PrF','Pr1','Pr2','PrS']
for idx, col in enumerate(prob_cols):
    out_df[col] = pred_prob[:, idx]

out_df['sentence_id'] = sid_list
out_df['word_in_sentence'] = wordpos_list

# 8. Merge with word map to get the actual word strings
out_df = out_df.merge(df_map, on=['sentence_id','word_in_sentence'])

# 9. Reorder columns
cols = ['word','sentence_id','word_in_sentence'] + cont_cols + prob_cols
out_df = out_df[cols]

# 10. Save to CSV
out_df.to_csv(OUT_CSV, index=False)
print(f"Saved word-based means CSV to {OUT_CSV}")
